In [6]:
root='/kaggle/working/'
import sys
module_path = root+'thesis/src/EDA'
sys.path.append(module_path)
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import seaborn as sns
from scipy import stats
import random
from scipy.stats import chi2
import statsmodels.api as sm
import matplotlib.pyplot as plt
from statsmodels.graphics.gofplots import qqplot
import pandas as pd
!pip install fastparquet
!pip install gdown

Questa è un analisi per capire su quanti utenti abbiamo un vero e proprio longitudinale di commenti. 
Si definisce longitudinale un dato se abbiamo almento 50 commenti, con un rate di 2 commenti per mese in media.

## Facebook

In [9]:
# Scarica i tre dataset che appartengono a Facebook
import gdown
url='https://drive.google.com/uc?id=1Y2lGWkcgo_IWHdWFh_Qcn0K74D_xQhvB'
output='facebook_news.csv'
gdown.download(url,output,quiet=False)

url='https://drive.google.com/uc?id=1KGH2s8sPnvXZc7_wVgTCoGTXPr3B7SCe'
output='facebook_brexit.csv'
gdown.download(url,output,quiet=False)

url='https://drive.google.com/uc?id=17q4X4JHYJBz1jF58XvYfbKqZLy-jtmzM'
output='facebook_vaccines.csv'
gdown.download(url,output,quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1Y2lGWkcgo_IWHdWFh_Qcn0K74D_xQhvB
From (redirected): https://drive.google.com/uc?id=1Y2lGWkcgo_IWHdWFh_Qcn0K74D_xQhvB&confirm=t&uuid=228dad60-15e0-49e1-a331-98711fa799d3
To: /kaggle/working/facebook_news.csv
100%|██████████| 1.84G/1.84G [00:18<00:00, 98.1MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1KGH2s8sPnvXZc7_wVgTCoGTXPr3B7SCe
From (redirected): https://drive.google.com/uc?id=1KGH2s8sPnvXZc7_wVgTCoGTXPr3B7SCe&confirm=t&uuid=708e1bb2-748b-427c-a4ef-35612566d704
To: /kaggle/working/facebook_brexit.csv
100%|██████████| 119M/119M [00:02<00:00, 50.0MB/s] 
Downloading...
From (original): https://drive.google.com/uc?id=17q4X4JHYJBz1jF58XvYfbKqZLy-jtmzM
From (redirected): https://drive.google.com/uc?id=17q4X4JHYJBz1jF58XvYfbKqZLy-jtmzM&confirm=t&uuid=9308b4da-2ab6-47d4-85b0-856918ec259a
To: /kaggle/working/facebook_vaccines.csv
100%|██████████| 599M/599M [00:15<00:00, 39.5MB/s] 


'facebook_vaccines.csv'

In [30]:
# Leggi i tre dataset con pandas
facebook_news = pd.read_csv('/kaggle/working/facebook_news.csv', dtype={'from_id': str})
facebook_vaccines_all_comments_scored = pd.read_csv('/kaggle/working/facebook_vaccines.csv', dtype={'from_id': str})
facebook_brexit_all_comments_scored = pd.read_csv('/kaggle/working/facebook_brexit.csv', dtype={'from_id': str})

# Etichetta e raggruppa i tre dataset
facebook_vaccines = facebook_vaccines_all_comments_scored.copy()
facebook_vaccines['topic'] = 'Vaccines'

facebook_brexit= facebook_brexit_all_comments_scored.copy()
facebook_brexit['topic'] = 'Brexit'

facebook_news = facebook_news.copy()
facebook_news['topic'] = 'News'

facebook = pd.concat([facebook_vaccines, facebook_brexit, facebook_news])
facebook.reset_index(drop=True, inplace=True)

In [70]:
import pandas as pd
import numpy as np

def gini_coefficient(values):
    """Calcola l'indice di concentrazione di Gini per una serie di valori."""
    sorted_values = np.sort(values)
    cumulative_values = np.cumsum(sorted_values)
    lorenz_curve = cumulative_values / cumulative_values[-1]
    gini_index = 1 - 2 * np.trapz(lorenz_curve, dx=1/len(lorenz_curve))
    return gini_index

def calculate_gini_for_posts(df):
    """Calcola l'indice di concentrazione di Gini per ogni post_id."""
    gini_results = []
    
    # Raggruppa i dati per post_id
    grouped = df.groupby('post_id')
    
    for post_id, group in grouped:
        # Calcola l'attività per ogni utente
        user_activity = group.groupby('from_id')['activity'].sum()
        # Calcola l'indice di Gini per questa attività
        gini_index = gini_coefficient(user_activity.values)
        gini_results.append({'post_id': post_id, 'gini_index': gini_index})
    
    return pd.DataFrame(gini_results)



# Calcola l'indice di Gini per ogni post_id
gini_df = calculate_gini_for_posts(facebook)
print(gini_df)


KeyError: 'Column not found: activity'

# Gini index of partecipation

Calcoliamo per ogni thread un coefficiente di Gini che misuri la concentrazione dell'attività degli utenti.